In [20]:
import numpy as np
import matplotlib.pyplot as plt
import test
%matplotlib tk

import blp_fft
# Example to plot the bases of the example lattices. Each point in the graph represent
# A chebyshev basis function

#One quadrant of a (nearly) regular hexagon
U = blp_fft.HexLattice(2)
U.plot_basis()

#One quadrant of the L_1 ball in 2d
U = blp_fft.PadLattice(10)
U.plot_basis()

#one quadrant of a rhombic dodecahedron, which is a stellation of an octahedron
U = blp_fft.BCCLattice(7)
U.plot_basis()

#one quadrant of a truncated octahedron
U = blp_fft.FCCLattice(6)
U.plot_basis()

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import test
import blp_fft as bf
%matplotlib tk

# Compare integration accuracy of the Padua and Hex lattices. (The hex points are not c)
x_0 = -.7
y_0 = .3

f = lambda x, y: 72*(x - x_0)*(y - y_0)/((1 + 3*(x_0 - x)**2 + 3*(y_0 - y)**2)**3)
I = 0

for i, j in np.ndindex((2, 2)):
    i = 2*i - 1
    j = 2*j - 1
    u = 1 + 3*(i - y_0)**2 + 3*(j - x_0)**2
    I += i*j/u

pad_pcount = []
pad_error  = []
for i in range(2, 150, 5):
    U = bf.PadLattice(i)
    U.eval_func(f)
    pad_pcount.append(U.get_point_count())
    pad_error.append(np.abs(I - U.get_integral()))
    
hex_pcount = []
hex_error  = []
for i in range(1, 19):
    U = bf.HexLattice(i)
    U.eval_func(f)
    hex_pcount.append(U.get_point_count())
    hex_error.append(np.abs(I - U.get_integral()))
    
plt.semilogy(np.sqrt(np.array(pad_pcount)), pad_error)
plt.semilogy(np.sqrt(np.array(hex_pcount)), hex_error)
plt.show()